In [1]:
# uncomment line below if lasio is not installed
# !pip install lasio

import os
import pandas as pd
import lasio
import re
import time
import gc

In [2]:
src_folder = r"E:\Downloads\mha0003\Boggess\Off Depth\Copied Data\Monitoring 5H\9H" # Change to folder containing the stages
files = os.listdir(src_folder)
target_folder = r"E:\Downloads\mha0003\Boggess\Off Depth\Copied Data\9H on 5H Output" # Change to output folder for keeping final CSVs
suffix = ".csv"

In [3]:
'''

1. Loops through each stage folder
2. Loops through each file, convert to readable LAS data and append in a Dataframe
3. Output CSV is stored in each stage folder

'''
t_start = time.time()

for stage in files[8:]:
    frames = []
    directory = os.path.join(src_folder, stage)
    las_files = os.listdir(directory)
    output_folder = os.path.join(target_folder, stage)
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)
    for file in las_files:
        las = lasio.read(os.path.join(src_folder, stage, file))
        match = re.findall(r"\d\d\d\d\d\d\d\d\d\d\d\d", str(file))
        df = las.df().reset_index()
        df.drop(df[df['SSTR'] > 25].index, inplace = True)
        df.drop(df[df['SSTR'] < 0].index, inplace = True)
        df.drop(df[df['DEPT'] > -8000].index, inplace = True)
        df.dropna(axis=0, inplace=True)
        df.drop(["RMS:1", "RMS:2", "RMS:3"], axis=1, inplace=True) # if error here, change RMS:1 to the correct header value based on output of df
        df.reset_index(inplace = True)
        df = df.apply(pd.to_numeric)
        df.insert(0, "Timestep", int(match[0]))
        frames.append(df)
    big_data = pd.concat(frames)
    big_data = big_data.loc[:, ~big_data.columns.str.contains('^Unnamed')]
    big_data.to_csv(os.path.join(output_folder, stage + suffix))
    del [[big_data]]
    gc.collect()
    big_data = pd.DataFrame()
    t_end = time.time()
    print("Time Taken for {} : {}".format(stage, t_end-t_start))
    t_start = time.time()

Time Taken for Stage 09 : 3039.0314762592316
